In [ ]:
import pickle
import gzip
import os, sys, errno
sys.path.insert(0, '../configuration')
sys.path.insert(0, '../utils')
sys.path.insert(0,"../io_funcs")
sys.path.insert(0,"../frontend")
from binary_io import BinaryIOCollection
from configuration import configuration
from utils import read_file_list, prepare_file_path_list
import time
import math

import subprocess
import logging
import numpy as np
#import gnumpy as gnp
# we need to explicitly import this in some cases, not sure why this doesn't get imported with numpy itself
import numpy.distutils.__config__
# and only after that can we import theano
import pdb
from matplotlib import pyplot as plt

In [2]:
cfg = configuration()
    # set up logging to use our custom class
    # logging.setLoggerClass(LoggerPlotter)
    # get a logger for this main function
logger = logging.getLogger("main")
config_file = "../../egs/blz/s1/conf/acoustic_blz3.conf"
config_file = os.path.abspath(config_file)
cfg.configure(config_file,False)
file_id_list = read_file_list(cfg.file_id_scp)
in_file_list_dict = {}
io_funcs = BinaryIOCollection()
for feature_name in list(cfg.in_dir_dict.keys()):
    in_file_list_dict[feature_name] = prepare_file_path_list(file_id_list, cfg.in_dir_dict[feature_name],
                                                             cfg.file_extension_dict[feature_name], False)

2019-04-16 23:19:54,526     INFO  configuration:           Paths:work has    user value /home/gyzhang/merlin/egs/blz/s1/experiments/blz3/acoustic_model
2019-04-16 23:19:54,528     INFO  configuration:           Paths:data has    user value /home/gyzhang/merlin/egs/blz/s1/experiments/blz3/acoustic_model/data
2019-04-16 23:19:54,530     INFO  configuration:     Paths:inter_data has    user value /home/gyzhang/merlin/egs/blz/s1/experiments/blz3/acoustic_model/inter_module
2019-04-16 23:19:54,532     INFO  configuration:           Paths:plot has    user value /home/gyzhang/merlin/egs/blz/s1/experiments/blz3/acoustic_model/plots
2019-04-16 23:19:54,533     INFO  configuration:       Paths:inp_feat has default value /home/gyzhang/merlin/egs/blz/s1/experiments/blz3/acoustic_model/inter_module/nn_no_silence_lab_norm_425
2019-04-16 23:19:54,534     INFO  configuration:       Paths:out_feat has default value /home/gyzhang/merlin/egs/blz/s1/experiments/blz3/acoustic_model/inter_module/nn_norm_mgc

2019-04-16 23:19:54,598     INFO  configuration: Labels:xpath_label_align has default value /home/gyzhang/merlin/egs/blz/s1/experiments/blz3/acoustic_model/data/label_state_align
2019-04-16 23:19:54,599     INFO  configuration: Labels:enforce_silence has    user value False
2019-04-16 23:19:54,600     INFO  configuration: Labels:remove_silence_using_binary_labels has default value False
2019-04-16 23:19:54,601     INFO  configuration: Labels:remove_silence_using_hts_labels has default value True
2019-04-16 23:19:54,602     INFO  configuration: Labels:precompile_xpaths has default value True
2019-04-16 23:19:54,603     INFO  configuration: Labels:iterate_over_frames has default value True
2019-04-16 23:19:54,604     INFO  configuration: Labels:appended_input_dim has default value 0
2019-04-16 23:19:54,605     INFO  configuration:     Data:buffer_size has    user value 200000
2019-04-16 23:19:54,606     INFO  configuration: Data:train_file_number has    user value 440
2019-04-16 23:19:54

2019-04-16 23:19:54,681     INFO  configuration:          Outputs:mgc has    user value 60
2019-04-16 23:19:54,682     INFO  configuration:         Outputs:dmgc has    user value 180
2019-04-16 23:19:54,683     INFO  configuration:          Outputs:vuv has default value 1
2019-04-16 23:19:54,684     INFO  configuration:          Outputs:lf0 has    user value 1
2019-04-16 23:19:54,685     INFO  configuration:         Outputs:dlf0 has    user value 3
2019-04-16 23:19:54,687     INFO  configuration:          Outputs:bap has    user value 1
2019-04-16 23:19:54,688     INFO  configuration:         Outputs:dbap has    user value 3
2019-04-16 23:19:54,689     INFO  configuration:          Outputs:cmp has default value 259
2019-04-16 23:19:54,690     INFO  configuration:    Outputs:stepw_dim has default value 55
2019-04-16 23:19:54,691     INFO  configuration:  Outputs:temp_sp_dim has default value 1025
2019-04-16 23:19:54,692     INFO  configuration:   Outputs:seglf0_dim has default value 7
2

2019-04-16 23:19:54,822     INFO  configuration:    Extensions:F0_ext has default value .F0
2019-04-16 23:19:54,825     INFO  configuration:  Extensions:Gain_ext has default value .Gain
2019-04-16 23:19:54,827     INFO  configuration:   Extensions:HNR_ext has default value .HNR
2019-04-16 23:19:54,829     INFO  configuration:   Extensions:LSF_ext has default value .LSF
2019-04-16 23:19:54,830     INFO  configuration: Extensions:LSFsource_ext has default value .LSFsource
2019-04-16 23:19:54,831     INFO  configuration:    Extensions:f0_ext has default value .f0
2019-04-16 23:19:54,832     INFO  configuration:  Extensions:gain_ext has default value .gain
2019-04-16 23:19:54,833     INFO  configuration:   Extensions:hnr_ext has default value .hnr
2019-04-16 23:19:54,834     INFO  configuration:   Extensions:lsf_ext has default value .lsf
2019-04-16 23:19:54,835     INFO  configuration:  Extensions:slsf_ext has default value .slsf
2019-04-16 23:19:54,836     INFO  configuration:   Extensio

In [ ]:
mean_dict=dict()
file_dict=dict()

In [3]:
global_mean_vector=np.zeros((1, 62))
global_std_vector=np.zeros((1,62))
all_frame_number = 0
all_feature_list=[]
all_file_mean_list=[]
all_file_95_list=[]
all_file_5_list=[]
# feature_dict=dict()
mgc_feature_list = []
for num, file_id in enumerate(file_id_list):
    # file_dict[file_id]=[]
    mean_feature_list=[]
    p95_list=[]
    p5_list=[]
    for feature_name in list(cfg.in_dir_dict.keys()):
        in_file_name = in_file_list_dict[feature_name][num]
        in_feature_dim = cfg.in_dimension_dict[feature_name]
        logger.info("preprocessing ")
        features, frame_number = io_funcs.load_binary_file_frame(in_file_name, in_feature_dim)
        if feature_name=="lf0":
            features=features[features>0]
            features=np.reshape(features,[-1,1])
        if feature_name=="mgc":
            mgc_feature_list.append(features)
            # pdb.set_trace()
        p5_list.extend(np.percentile(features,5,axis=0))
        p95_list.extend(np.percentile(features,95,axis=0))
        mean_feature_list.extend(np.mean(features,0))
        # feature_dict[feature_name]=np.mean(features,1)
        # file_dict[file_id].append(feature_dict)
    all_file_mean_list.append(mean_feature_list)
    all_file_95_list.append(p95_list)
    all_file_5_list.append(p5_list)
mgc_features=np.concatenate(mgc_feature_list,axis=0)
global_mean_vector = np.mean(mgc_features,axis=0)
global_std_vector = np.std(mgc_features,axis=0)
global_p5_vector=np.percentile(mgc_features,5,axis=0)
global_p95_vector=np.percentile(mgc_features,95,axis=0)
all_file_mean_array=np.array(all_file_mean_list)
all_file_5_array=np.array(all_file_5_list)
all_file_95_array=np.array(all_file_95_list)

% **shows feature**
utterance's mgc features

In [4]:
for utterance_index in range(len(file_id_list)):
    for dim_index in range(60):
        in_file_name = in_file_list_dict['mgc'][utterance_index]
        in_feature_dim = cfg.in_dimension_dict['mgc']
        features, frame_number = io_funcs.load_binary_file_frame(in_file_name, in_feature_dim)
        features = features[:,dim_index]
        mean=np.mean(features)
        std=np.std(features)
        p5=np.percentile(features,5)
        p95=np.percentile(features,95)
      #  print((p95-p5)/std,(p95-p5)/global_std_vector[dim_index])
        right_side = (p95-mean)/std-(p95-global_mean_vector[dim_index])/global_std_vector[dim_index]
        left_side = (p5-mean)/std-(p5-global_mean_vector[dim_index])/global_std_vector[dim_index]
        intervel = (p95-p5)/std-(p95-p5)/global_std_vector[dim_index]
        if abs(right_side)>1 or  abs(left_side)>1:
            print(utterance_index,(p95-p5)/std,dim_index,right_side,left_side,file_id_list[utterance_index])
        if abs(intervel)>1:
            print(utterance_index,intervel,file_id_list[utterance_index])
      #  print(std,global_std_vector[dim_index],mean,global_mean_vector[dim_index])

36 3.2477050117280943 15 0.4676430185826713 1.0039606661901568 100039
47 3.2401607936982586 6 0.8532535582060226 1.1809039141796915 100050
47 3.301623120553555 7 -1.1856131269754635 -0.8279163798662482 100050
61 3.218692326947174 15 0.3617345538176995 1.0672085605112802 100064
64 3.2657596850990713 13 0.9439270385608216 1.0234956185444817 100068
64 3.2261282848273765 14 -1.167640792622456 -0.6607967424812511 100068
102 3.2413252733364155 22 0.40361146350529864 1.1370057031608358 100107
112 3.163273732935558 22 0.21663250100413012 1.089898717200815 100117
116 3.2303332508164053 11 1.032658611365739 0.5875315648007668 100121
126 3.3302566052441467 13 0.6905633354963914 1.019434092691556 100132
162 3.31106973010736 5 0.983844417955281 1.0618002239433568 100172
165 3.2504610337536124 8 1.1440160895812412 0.6628017576920804 100175
166 3.272429978616602 3 -0.9947869268452982 -1.0352933810092608 100176
171 3.294257841286415 8 -1.0927398779710396 -0.606043579467758 100181
176 3.279925115851874

In [12]:
file_id_list[112]

'100117'

In [6]:
global_mean_vector

array([-5.371039  ,  1.7104896 ,  0.15252385,  0.20157169,  0.1266414 ,
        0.22784942,  0.04235072,  0.04974031, -0.0656646 ,  0.06131163,
       -0.23542984,  0.24174117, -0.17643562,  0.07422392, -0.05731717,
        0.0297751 , -0.03881345,  0.02696932, -0.06133305,  0.07005785,
       -0.09373155,  0.08294727, -0.09484902,  0.05836009, -0.07114811,
        0.04768491, -0.04852841,  0.06179808, -0.07179679,  0.06587469,
       -0.05206925,  0.05720416, -0.0441705 ,  0.06519852, -0.05018669,
        0.05490638, -0.0533908 ,  0.05505326, -0.0464406 ,  0.04896504,
       -0.04478316,  0.04003001, -0.03804275,  0.03899866, -0.04049207,
        0.04126386, -0.03965253,  0.03458681, -0.03177643,  0.02886853,
       -0.02599358,  0.0251983 , -0.02670573,  0.02762241, -0.02835807,
        0.02850229, -0.02781359,  0.02531525, -0.02235571,  0.01872815],
      dtype=float32)

In [7]:
all_file_mean_array[244,0:10]

array([-5.4769564 ,  1.636708  ,  0.31434533,  0.01890735,  0.13687642,
        0.24861178,  0.01018146, -0.0074994 , -0.03746642,  0.04517589],
      dtype=float32)

In [8]:
all_file_mean_array[0,1:3]

array([1.6443429 , 0.02213725], dtype=float32)

In [ ]:
utterance_index=4
dim_index = 56
in_file_name = in_file_list_dict['mgc'][utterance_index]
in_feature_dim = cfg.in_dimension_dict['mgc']
features, frame_number = io_funcs.load_binary_file_frame(in_file_name, in_feature_dim)

% frist dimension of mgc

In [ ]:
features = features[:,dim_index]
mean=np.mean(features)
std=np.std(features)
p5=np.percentile(features,5)
p95=np.percentile(features,95)

In [ ]:
print(std,mean,global_std_vector[dim_index],global_mean_vector[dim_index])

In [ ]:
n, bins, patches = plt.hist(x=(features-mean)/std, bins="auto", color='#0504aa',
                            alpha=0.7, rwidth=0.85)
plt.show()

In [ ]:
n, bins, patches = plt.hist(x=mgc_features[:,56], bins="auto", color='#0504aa', alpha=0.7, rwidth=0.85)
plt.title('57th dimension of MC feature')

In [ ]:
plt.show()